In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['airlines.csv', 'flights.csv', 'airports.csv']


In [3]:
from sklearn.model_selection import GridSearchCV

In [4]:
import pandas as pd, numpy as np, time
from sklearn.model_selection import train_test_split

data = pd.read_csv("../input/flights.csv")
data = data.sample(frac = 0.1, random_state=10)

data = data[["MONTH","DAY","DAY_OF_WEEK","AIRLINE","FLIGHT_NUMBER","DESTINATION_AIRPORT",
                 "ORIGIN_AIRPORT","AIR_TIME", "DEPARTURE_TIME","DISTANCE","ARRIVAL_DELAY"]]
data.dropna(inplace=True)

data["ARRIVAL_DELAY"] = (data["ARRIVAL_DELAY"]>10)*1

cols = ["AIRLINE","FLIGHT_NUMBER","DESTINATION_AIRPORT","ORIGIN_AIRPORT"]
for item in cols:
    data[item] = data[item].astype("category").cat.codes +1
 
train, test, y_train, y_test = train_test_split(data.drop(["ARRIVAL_DELAY"], axis=1), data["ARRIVAL_DELAY"],
random_state=10, test_size=0.25)

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
import xgboost as xgb
from sklearn import metrics

def auc(m, train, test): 
    return (metrics.roc_auc_score(y_train,m.predict_proba(train)[:,1]),
                            metrics.roc_auc_score(y_test,m.predict_proba(test)[:,1]))

# Parameter Tuning
model = xgb.XGBClassifier()
param_dist = {"max_depth": [10],
              "min_child_weight" : [1,3,6],
              "n_estimators": [200],
              "learning_rate": [0.05, 0.1,0.16],}
grid_search = GridSearchCV(model, param_grid=param_dist, cv = 3, 
                                   verbose=10, n_jobs=-1)
grid_search.fit(train, y_train)

grid_search.best_estimator_

model = xgb.XGBClassifier(max_depth=7, min_child_weight=1,  n_estimators=200,
                          n_jobs=-1 , verbose=1,learning_rate=0.16)
model.fit(train,y_train)

auc(model, train, test)

Fitting 3 folds for each of 9 candidates, totalling 27 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:  6.9min
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed: 10.2min
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed: 16.4min
[Parallel(n_jobs=-1)]: Done  23 out of  27 | elapsed: 19.9min remaining:  3.5min
[Parallel(n_jobs=-1)]: Done  27 out of  27 | elapsed: 22.7min finished


(0.8073006385510831, 0.7585950899263778)

In [8]:
import lightgbm as lgb
from sklearn import metrics

def auc2(m, train, test): 
    return (metrics.roc_auc_score(y_train,m.predict(train)),
                            metrics.roc_auc_score(y_test,m.predict(test)))

# lg = lgb.LGBMClassifier(silent=False)
# param_dist = {"max_depth": [4],
#               "learning_rate" : [0.1],
#               "num_leaves": [300,900],
#               "n_estimators": [100]
#              }
# grid_search = GridSearchCV(lg, n_jobs=-1, param_grid=param_dist, cv = 3, scoring="roc_auc", verbose=5)
# grid_search.fit(train,y_train)
# grid_search.best_estimator_

d_train = lgb.Dataset(train, label=y_train)
params = {"max_depth": 7, "learning_rate" : 0.1, "num_leaves": 900,  "n_estimators": 200}

# Without Categorical Features
model2 = lgb.train(params, d_train)
auc2(model2, train, test)

# #With Catgeorical Features
# cate_features_name = ["MONTH","DAY","DAY_OF_WEEK","AIRLINE","DESTINATION_AIRPORT",
#                  "ORIGIN_AIRPORT"]
# model2 = lgb.train(params, d_train, categorical_feature = cate_features_name)
# auc2(model2, train, test)

/opt/conda/lib/python3.6/site-packages/lightgbm/engine.py:113: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


(0.7879701276144686, 0.7535978962749439)

In [9]:
import catboost as cb
cat_features_index = [0,1,2,3,4,5,6]

def auc(m, train, test): 
    return (metrics.roc_auc_score(y_train,m.predict_proba(train)[:,1]),
                            metrics.roc_auc_score(y_test,m.predict_proba(test)[:,1]))

# params = {'depth': [4],
#           'learning_rate' : [0.1],
#          'l2_leaf_reg': [1,4,9],
#          'iterations': [100]}
# cb = cb.CatBoostClassifier()
# cb_model = GridSearchCV(cb, params, scoring="roc_auc", cv = 3)
# cb_model.fit(train, y_train)

print("With Categorical features")
clf = cb.CatBoostClassifier(eval_metric="AUC", depth=7, iterations= 200, l2_leaf_reg= 9, learning_rate= 0.15)
clf.fit(train,y_train)
auc(clf, train, test)
# print("With Categorical features")
# clf = cb.CatBoostClassifier(eval_metric="AUC",one_hot_max_size=31, \
#                             depth=7, iterations= 100, l2_leaf_reg= 9, learning_rate= 0.15)
# clf.fit(train,y_train, cat_features= cat_features_index)
# auc(clf, train, test)

With Categorical features
0:	total: 160ms	remaining: 31.8s
1:	total: 259ms	remaining: 25.7s
2:	total: 347ms	remaining: 22.8s
3:	total: 451ms	remaining: 22.1s
4:	total: 553ms	remaining: 21.6s
5:	total: 649ms	remaining: 21s
6:	total: 755ms	remaining: 20.8s
7:	total: 856ms	remaining: 20.5s
8:	total: 960ms	remaining: 20.4s
9:	total: 1.05s	remaining: 20s
10:	total: 1.15s	remaining: 19.8s
11:	total: 1.25s	remaining: 19.6s
12:	total: 1.35s	remaining: 19.5s
13:	total: 1.45s	remaining: 19.2s
14:	total: 1.54s	remaining: 19.1s
15:	total: 1.64s	remaining: 18.9s
16:	total: 1.74s	remaining: 18.7s
17:	total: 1.83s	remaining: 18.5s
18:	total: 1.93s	remaining: 18.3s
19:	total: 2.02s	remaining: 18.2s
20:	total: 2.12s	remaining: 18.1s
21:	total: 2.21s	remaining: 17.9s
22:	total: 2.31s	remaining: 17.8s
23:	total: 2.4s	remaining: 17.6s
24:	total: 2.49s	remaining: 17.5s
25:	total: 2.59s	remaining: 17.4s
26:	total: 2.69s	remaining: 17.2s
27:	total: 2.78s	remaining: 17.1s
28:	total: 2.88s	remaining: 17s
29:	t

(0.7451022141205017, 0.73456850844974)